# Build Sample Dataset

## Setup

In [1]:
# Setup and Imports
import json
import pandas as pd

## Load Datasets

In [2]:
prm800k = pd.read_csv("datasets/prm800k-test-valid-data.csv")
prm800k

,labeler,timestamp,generation,is_quality_control_question,is_initial_screening_question,question,label,total_steps,total_steps_have_neg_1,steps_neg_1,total_neg_1_sequence_from_last_step,total_steps_have_zero,steps_zero,total_steps_have_pos_1_or_human,steps_pos_1_or_human,total_steps_have_combination,finish_reason
0,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T14:37:13.296218,NaN,False,False,{'problem': 'There are an infinite number of v...,"{'steps': [{'completions': [{'text': ""Let's se...",34,22,"[2, 3, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",3,14,"[2, 3, 4, 6, 7, 9, 11, 13, 18, 20, 24, 26, 27,...",34,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",11,solution
1,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T13:26:58.414691,NaN,False,False,{'problem': 'When rolling a certain unfair six...,"{'steps': [{'completions': [{'text': ""Well, le...",35,18,"[2, 3, 4, 6, 8, 9, 12, 13, 14, 17, 18, 19, 20,...",0,11,"[3, 4, 8, 9, 12, 13, 15, 18, 21, 28, 31]",35,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",10,solution
2,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-31T14:39:30.588403,NaN,False,False,{'problem': 'Find all solutions to\n\\[\\sin \...,"{'steps': [{'completions': [{'text': ""Let's us...",34,13,"[1, 2, 6, 7, 8, 14, 16, 18, 29, 30, 31, 32, 34]",1,7,"[2, 7, 8, 14, 16, 33, 34]",34,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution
3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-29T07:48:01.714041,NaN,False,False,{'problem': 'The solutions of the equation $z^...,{'steps': [{'completions': [{'text': 'There is...,40,17,"[2, 6, 8, 9, 11, 12, 16, 17, 18, 20, 25, 30, 3...",0,8,"[2, 3, 16, 17, 18, 20, 30, 40]",40,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution
4,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-22T20:02:50.866783,NaN,False,False,{'problem': 'A sequence $(a_n)$ is defined as ...,"{'steps': [{'completions': [{'text': ""So we're...",36,9,"[5, 9, 10, 11, 12, 13, 14, 15, 16]",0,8,"[5, 6, 9, 11, 12, 14, 16, 35]",36,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,a2e09d3a-2da6-4094-89ca-e92125f6aefd,2022-11-11T23:53:32.446394,6.0,False,False,{'problem': 'Find the product of $6_8 \\cdot 7...,{'steps': [{'completions': [{'text': 'I notice...,15,0,[],0,0,[],15,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution
538,2600b6ef-d904-43e8-ba92-de2fec65e275,2022-11-11T23:43:18.235408,6.0,False,False,"{'problem': 'Compute $\\dbinom{31}{28}$.', 'gr...",{'steps': [{'completions': [{'text': 'To compu...,6,1,[6],1,0,[],6,"[1, 2, 3, 4, 5, 6]",0,solution
539,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:24:31.318743,6.0,False,False,"{'problem': ""Alice and Bob are playing a game....",{'steps': [{'completions': [{'text': 'I need t...,16,0,[],0,0,[],16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution
540,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:19:53.088201,6.0,False,False,{'problem': 'In how many ways can $7$ people s...,{'steps': [{'completions': [{'text': 'This is ...,16,0,[],0,0,[],16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution


## Build Sample Dataset

### Functions

In [3]:
import json
import ast

def string_to_dict(input_string):
    input_string = input_string.strip()

    try:
        json_string = input_string.replace("'", '"')
        json_string = json_string.replace('True', 'true').replace('False', 'false').replace('None', 'null')
        return json.loads(json_string)
    except (json.JSONDecodeError, ValueError):
        pass
    
    try:
        return ast.literal_eval(input_string)
    except (ValueError, SyntaxError):
        pass
    
    try:
        if input_string.strip().startswith('{') and input_string.strip().endswith('}'):
            return eval(input_string)
    except:
        pass

    raise ValueError(f"Cannot parse string as dictionary: {input_string[:100]}...")

In [4]:
def get_ground_truth_steps(steps):
    ground_truth_steps = []
    steps_list = ast.literal_eval(steps) if isinstance(steps, str) else steps
    for step in steps_list:
        if step['human_completion'] != None:
            ground_truth_steps.append(step['human_completion']['text'])
        else:
            for completion in step['completions']: 
                if completion['rating'] == 1:
                    ground_truth_steps.append(completion['text']) 
                    break  # Break after finding first rating=1 completion
    return ground_truth_steps

In [5]:
def process_data(df, function):
    results = []
    counter = 1
    for index, row in df.iterrows():
        try:
            question = string_to_dict(row['question'])
            data = string_to_dict(row['label'])
            steps = data['steps']
            new_columns = function(steps)

            new_result = {
                "id": counter,
                "labeler": row["labeler"],
                "timestamp": row["timestamp"],
                "problem": question["problem"],
                "ground_truth_answer": question["ground_truth_answer"],
                "total_steps": row["total_steps"],
                "ground_truth_steps": get_ground_truth_steps(steps),
                **new_columns,
            }
            results.append(new_result)
            counter += 1
        
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue
    
    # Create DataFrame from results
    results_df = pd.DataFrame(results)
    return results_df

In [6]:
def get_steps_info(steps):
    neg_1 = 0
    zero = 0
    pos_1 = 0

    for step in steps:
        match step["rating"]:
            case -1:
                neg_1 += 1
            case 0:
                zero += 1
            case 1:
                pos_1 += 1
    
    steps_info = {
        "neg_1": neg_1,
        "zero": zero,
        "pos_1": pos_1
    }

    return steps_info

In [7]:
def get_total_steps_sum_each_rating(df):
    result = {
        "total_neg_1": df['neg_1'].sum(),
        "total_zero": df['zero'].sum(),
        "total_pos_1": df['pos_1'].sum()
    }
    return result

In [8]:
def drop_rows_where_pos1_equals_total_steps(df):
    return df[df["pos_1"] != df["total_steps"]].copy()

### [ALGO 1] Merge Completions in Each Steps to Index 0

In [9]:
def merge_completions_to_idx_0(steps):
    new_steps = []

    # ALGO: get first completion in every steps
    for step_no, step in enumerate(steps):
        collect_step = step['completions'][0]
        new_steps.append(collect_step)

    steps_info = get_steps_info(new_steps)
    result = {
        "steps": new_steps,
        **steps_info
    }
    return result

In [10]:
df_option_1 = process_data(prm800k, merge_completions_to_idx_0)
df_option_1.to_csv("datasets/sample/prm800k-01-first-completion.csv", index=False)

In [11]:
df_option_1

,id,labeler,timestamp,problem,ground_truth_answer,total_steps,ground_truth_steps,steps,neg_1,zero,pos_1
0,1,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T14:37:13.296218,There are an infinite number of vectors $\math...,\begin{pmatrix} -7 \\ 16 \\ 5 \end{pmatrix},34,[Let's set $\mathbf{v} = \begin{pmatrix} a \\ ...,[{'text': 'Let's set $\mathbf{v} = \begin{pmat...,15,10,9
1,2,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T13:26:58.414691,When rolling a certain unfair six-sided die wi...,29,35,"[Well, let's think about this for a moment. Wh...","[{'text': 'Well, let's think about this for a ...",11,8,16
2,3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-31T14:39:30.588403,Find all solutions to\n\[\sin \left( \tan^{-1}...,3 \pm 2 \sqrt{2},34,"[Let's set $y = \tan^{-1} x$., Then $\cot^{-1}...",[{'text': 'Let's use the identity $\sin(A+B)=\...,8,4,22
3,4,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-29T07:48:01.714041,The solutions of the equation $z^4+4z^3i-6z^2-...,11,40,[There is a formula for the area of any convex...,[{'text': 'There is a formula for the area of ...,14,5,21
4,5,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-22T20:02:50.866783,A sequence $(a_n)$ is defined as follows:\n\[a...,-1,36,[So we're given that $a_{i + 1} = \frac{1}{1 -...,[{'text': 'So we're given that $a_{i + 1} = \f...,5,6,25
...,...,...,...,...,...,...,...,...,...,...,...
537,538,a2e09d3a-2da6-4094-89ca-e92125f6aefd,2022-11-11T23:53:32.446394,Find the product of $6_8 \cdot 7_8.$ Express y...,52_8,15,[I notice that the problem is asking me to mul...,[{'text': 'I notice that the problem is asking...,0,0,15
538,539,2600b6ef-d904-43e8-ba92-de2fec65e275,2022-11-11T23:43:18.235408,Compute $\dbinom{31}{28}$.,4495,6,"[To compute $\dbinom{31}{28}$, I need to recal...","[{'text': 'To compute $\dbinom{31}{28}$, I nee...",1,0,5
539,540,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:24:31.318743,Alice and Bob are playing a game. Alice starts...,\frac{2}{3},16,[I need to find the probability that Alice win...,[{'text': 'I need to find the probability that...,0,0,16
540,541,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:19:53.088201,In how many ways can $7$ people sit around a r...,144,16,[This is a counting problem that involves perm...,[{'text': 'This is a counting problem that inv...,0,0,16


In [12]:
# Calculate Total neg_1, zero, pos_1
get_total_steps_sum_each_rating(df_option_1)

{'total_neg_1': np.int64(253),
 'total_zero': np.int64(154),
 'total_pos_1': np.int64(5520)}

Result: Still Not Good, Total Pos 1 Too Much, Many of Datas Only Pos 1 No Neg 1 & Zero

### [ALGO 2] Real Condition - For Early Step (<30%) Use 1, Then After It Random, If Found -1 The Use -1/0 After It

In [13]:
import random

def algo_2(steps):
    new_steps = []
    t1 = len(steps) / 3

    # ALGO: get first completion in every steps
    counter = 1
    latest_step = 1
    for step_no, step in enumerate(steps):
        collect_step = step['completions'][0]
        rating = collect_step["rating"]

        if counter <= t1:
            # CASE <= 33%
            if rating != 1:
                flag = False
                for idx, completion in enumerate(step['completions']):
                    if completion["rating"] == 1:
                        collect_step = completion
                        flag = True
                        break
                if not flag:
                    collect_step = step["human_completion"]
                    collect_step["rating"] = 1
            new_steps.append(collect_step)
            counter += 1
            continue
        
        # After 33%
        if latest_step == -1:
            for idx, completion in enumerate(step['completions']):
                if completion["rating"] == -1:
                    collect_step = completion
                    break
        
            # Berusaha -1, kalau ngga ketemu pakai default (first completion)
            new_steps.append(collect_step)
            continue
        
        total_completions = len(step['completions'])
        random_completion = step['completions'][random.randint(0, total_completions-1)]
        new_steps.append(random_completion)
        latest_step = random_completion['rating']

    steps_info = get_steps_info(new_steps)
    result = {
        "steps": new_steps,
        **steps_info
    }
    return result

In [14]:
df_option_2 = process_data(prm800k, algo_2)
df_option_2.to_csv("datasets/sample/prm800k-02-algo2.csv", index=False)

In [15]:
df_option_2

,id,labeler,timestamp,problem,ground_truth_answer,total_steps,ground_truth_steps,steps,neg_1,zero,pos_1
0,1,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T14:37:13.296218,There are an infinite number of vectors $\math...,\begin{pmatrix} -7 \\ 16 \\ 5 \end{pmatrix},34,[Let's set $\mathbf{v} = \begin{pmatrix} a \\ ...,[{'text': 'Let's set $\mathbf{v} = \begin{pmat...,15,1,18
1,2,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T13:26:58.414691,When rolling a certain unfair six-sided die wi...,29,35,"[Well, let's think about this for a moment. Wh...","[{'text': 'Well, let's think about this for a ...",12,1,22
2,3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-31T14:39:30.588403,Find all solutions to\n\[\sin \left( \tan^{-1}...,3 \pm 2 \sqrt{2},34,"[Let's set $y = \tan^{-1} x$., Then $\cot^{-1}...","[{'text': 'Let's set $y = \tan^{-1} x$.', 'rat...",7,2,25
3,4,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-29T07:48:01.714041,The solutions of the equation $z^4+4z^3i-6z^2-...,11,40,[There is a formula for the area of any convex...,[{'text': 'There is a formula for the area of ...,11,1,28
4,5,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-22T20:02:50.866783,A sequence $(a_n)$ is defined as follows:\n\[a...,-1,36,[So we're given that $a_{i + 1} = \frac{1}{1 -...,[{'text': 'So we're given that $a_{i + 1} = \f...,4,1,31
...,...,...,...,...,...,...,...,...,...,...,...
537,538,a2e09d3a-2da6-4094-89ca-e92125f6aefd,2022-11-11T23:53:32.446394,Find the product of $6_8 \cdot 7_8.$ Express y...,52_8,15,[I notice that the problem is asking me to mul...,[{'text': 'I notice that the problem is asking...,0,0,15
538,539,2600b6ef-d904-43e8-ba92-de2fec65e275,2022-11-11T23:43:18.235408,Compute $\dbinom{31}{28}$.,4495,6,"[To compute $\dbinom{31}{28}$, I need to recal...","[{'text': 'To compute $\dbinom{31}{28}$, I nee...",0,0,6
539,540,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:24:31.318743,Alice and Bob are playing a game. Alice starts...,\frac{2}{3},16,[I need to find the probability that Alice win...,[{'text': 'I need to find the probability that...,0,0,16
540,541,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:19:53.088201,In how many ways can $7$ people sit around a r...,144,16,[This is a counting problem that involves perm...,[{'text': 'This is a counting problem that inv...,0,0,16


In [16]:
# Calculate Total neg_1, zero, pos_1
get_total_steps_sum_each_rating(df_option_2)

{'total_neg_1': np.int64(209),
 'total_zero': np.int64(53),
 'total_pos_1': np.int64(5665)}

Result: Still Not Good, Total Pos 1 Too Much, Many of Datas Only Pos 1 No Neg 1 & Zero

In [17]:
# Try To Drop All Pos 1
clean_df_option_2 = drop_rows_where_pos1_equals_total_steps(df_option_2)

In [18]:
clean_df_option_2

,id,labeler,timestamp,problem,ground_truth_answer,total_steps,ground_truth_steps,steps,neg_1,zero,pos_1
0,1,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T14:37:13.296218,There are an infinite number of vectors $\math...,\begin{pmatrix} -7 \\ 16 \\ 5 \end{pmatrix},34,[Let's set $\mathbf{v} = \begin{pmatrix} a \\ ...,[{'text': 'Let's set $\mathbf{v} = \begin{pmat...,15,1,18
1,2,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T13:26:58.414691,When rolling a certain unfair six-sided die wi...,29,35,"[Well, let's think about this for a moment. Wh...","[{'text': 'Well, let's think about this for a ...",12,1,22
2,3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-31T14:39:30.588403,Find all solutions to\n\[\sin \left( \tan^{-1}...,3 \pm 2 \sqrt{2},34,"[Let's set $y = \tan^{-1} x$., Then $\cot^{-1}...","[{'text': 'Let's set $y = \tan^{-1} x$.', 'rat...",7,2,25
3,4,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-29T07:48:01.714041,The solutions of the equation $z^4+4z^3i-6z^2-...,11,40,[There is a formula for the area of any convex...,[{'text': 'There is a formula for the area of ...,11,1,28
4,5,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-22T20:02:50.866783,A sequence $(a_n)$ is defined as follows:\n\[a...,-1,36,[So we're given that $a_{i + 1} = \frac{1}{1 -...,[{'text': 'So we're given that $a_{i + 1} = \f...,4,1,31
5,6,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-08-03T08:31:13.102743,"If $\arccos x + \arccos 2x + \arccos 3x = \pi,...",27,30,[Let's use the identity $\cos(\alpha + \beta) ...,[{'text': 'Let's use the identity $\cos(\alpha...,6,1,23
6,7,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-24T20:15:48.777873,In how many ways can $7$ people sit around a r...,144,15,"[Let's first seat Pierre, Rosa, and Thomas., Y...","[{'text': 'Let's first seat Pierre, Rosa, and ...",3,1,11
7,8,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-29T17:59:08.576739,Let\n\[f(x) = (\arccos x)^2 + (\arcsin x)^2.\]...,"\left[ \frac{\pi^2}{8}, \frac{5 \pi^2}{4} \right]",26,"[Let's use a substitution., Ok, let $u=\arccos...","[{'text': 'Let's use a substitution.', 'rating...",4,2,20
8,9,d8aa7923-b970-45e1-9734-e4a7f6c4a7db,2022-08-02T17:50:24.697088,"For every positive integer $n$, let $\text{mod...",1,52,[I think we can solve this by just computing a...,[{'text': 'I think we can solve this by just c...,6,4,42
9,10,e90a38f3-3135-4465-87af-3e6322e3d772,2022-08-05T23:00:06.860704,"Let $a,$ $b,$ and $c$ be distinct real numbers...",0,17,[Let's write out the polynomial so that we can...,[{'text': 'Let's write out the polynomial so t...,5,2,10


In [19]:
get_total_steps_sum_each_rating(clean_df_option_2)

{'total_neg_1': np.int64(209),
 'total_zero': np.int64(53),
 'total_pos_1': np.int64(936)}

Result: Good, Natural PRM800K
| Label | Jumlah | Persentase |
| ----- | ------ | ---------- |
| -1    | 211    | \~17.3%    |
| 0     | 54     | \~4.4%     |
| +1    | 955    | \~78.3%    |

In [20]:
clean_df_option_2.to_csv("datasets/sample/prm800k-02-algo2-clean.csv", index=False)

### [ALGO 3] Real Condition - Random Rating From Early, If Found -1 The Use -1/0 After It

In [21]:
import random

def algo_3(steps):
    new_steps = []

    latest_step = 1
    for step_no, step in enumerate(steps):
        collect_step = step['completions'][0]

        # From Early
        if latest_step == -1:
            for idx, completion in enumerate(step['completions']):
                if completion["rating"] == -1:
                    collect_step = completion
                    break
        
            # Berusaha -1, kalau ngga ketemu pakai default (first completion)
            new_steps.append(collect_step)
            continue
        
        total_completions = len(step['completions'])
        random_completion = step['completions'][random.randint(0, total_completions-1)]
        new_steps.append(random_completion)
        latest_step = random_completion['rating']

    steps_info = get_steps_info(new_steps)
    result = {
        "steps": new_steps,
        **steps_info
    }
    return result

In [22]:
df_option_3 = process_data(prm800k, algo_3)
df_option_3.to_csv("datasets/sample/prm800k-02-algo3.csv", index=False)

In [23]:
df_option_3

,id,labeler,timestamp,problem,ground_truth_answer,total_steps,ground_truth_steps,steps,neg_1,zero,pos_1
0,1,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T14:37:13.296218,There are an infinite number of vectors $\math...,\begin{pmatrix} -7 \\ 16 \\ 5 \end{pmatrix},34,[Let's set $\mathbf{v} = \begin{pmatrix} a \\ ...,[{'text': 'Let's set $\mathbf{v} = \begin{pmat...,22,3,9
1,2,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T13:26:58.414691,When rolling a certain unfair six-sided die wi...,29,35,"[Well, let's think about this for a moment. Wh...","[{'text': 'Well, let's think about this for a ...",18,1,16
2,3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-31T14:39:30.588403,Find all solutions to\n\[\sin \left( \tan^{-1}...,3 \pm 2 \sqrt{2},34,"[Let's set $y = \tan^{-1} x$., Then $\cot^{-1}...","[{'text': 'Let's call $\tan^{-1}(x)=a$.', 'rat...",12,1,21
3,4,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-29T07:48:01.714041,The solutions of the equation $z^4+4z^3i-6z^2-...,11,40,[There is a formula for the area of any convex...,[{'text': 'There is a formula for the area of ...,16,2,22
4,5,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-22T20:02:50.866783,A sequence $(a_n)$ is defined as follows:\n\[a...,-1,36,[So we're given that $a_{i + 1} = \frac{1}{1 -...,[{'text': 'So we're given that $a_{i + 1} = \f...,7,4,25
...,...,...,...,...,...,...,...,...,...,...,...
537,538,a2e09d3a-2da6-4094-89ca-e92125f6aefd,2022-11-11T23:53:32.446394,Find the product of $6_8 \cdot 7_8.$ Express y...,52_8,15,[I notice that the problem is asking me to mul...,[{'text': 'I notice that the problem is asking...,0,0,15
538,539,2600b6ef-d904-43e8-ba92-de2fec65e275,2022-11-11T23:43:18.235408,Compute $\dbinom{31}{28}$.,4495,6,"[To compute $\dbinom{31}{28}$, I need to recal...","[{'text': 'To compute $\dbinom{31}{28}$, I nee...",1,0,5
539,540,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:24:31.318743,Alice and Bob are playing a game. Alice starts...,\frac{2}{3},16,[I need to find the probability that Alice win...,[{'text': 'I need to find the probability that...,0,0,16
540,541,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:19:53.088201,In how many ways can $7$ people sit around a r...,144,16,[This is a counting problem that involves perm...,[{'text': 'This is a counting problem that inv...,0,0,16


In [24]:
# Try To Drop All Pos 1
clean_df_option_3 = drop_rows_where_pos1_equals_total_steps(df_option_3)

In [25]:
clean_df_option_3

,id,labeler,timestamp,problem,ground_truth_answer,total_steps,ground_truth_steps,steps,neg_1,zero,pos_1
0,1,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T14:37:13.296218,There are an infinite number of vectors $\math...,\begin{pmatrix} -7 \\ 16 \\ 5 \end{pmatrix},34,[Let's set $\mathbf{v} = \begin{pmatrix} a \\ ...,[{'text': 'Let's set $\mathbf{v} = \begin{pmat...,22,3,9
1,2,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T13:26:58.414691,When rolling a certain unfair six-sided die wi...,29,35,"[Well, let's think about this for a moment. Wh...","[{'text': 'Well, let's think about this for a ...",18,1,16
2,3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-31T14:39:30.588403,Find all solutions to\n\[\sin \left( \tan^{-1}...,3 \pm 2 \sqrt{2},34,"[Let's set $y = \tan^{-1} x$., Then $\cot^{-1}...","[{'text': 'Let's call $\tan^{-1}(x)=a$.', 'rat...",12,1,21
3,4,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-29T07:48:01.714041,The solutions of the equation $z^4+4z^3i-6z^2-...,11,40,[There is a formula for the area of any convex...,[{'text': 'There is a formula for the area of ...,16,2,22
4,5,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-22T20:02:50.866783,A sequence $(a_n)$ is defined as follows:\n\[a...,-1,36,[So we're given that $a_{i + 1} = \frac{1}{1 -...,[{'text': 'So we're given that $a_{i + 1} = \f...,7,4,25
...,...,...,...,...,...,...,...,...,...,...,...
439,440,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-28T08:12:20.344377,Find the product $CD$ of the integers $C$ and ...,-5,17,[I think the first step here is to find the le...,[{'text': 'I think the first step here is to f...,3,0,14
441,442,d8aa7923-b970-45e1-9734-e4a7f6c4a7db,2022-07-31T22:47:06.498122,What real values of $x$ are not in the domain ...,-4,31,[To find values of $x$ that are not in the dom...,[{'text': 'To find values of $x$ that are not ...,4,0,27
443,444,d8aa7923-b970-45e1-9734-e4a7f6c4a7db,2022-07-24T10:40:50.685197,How many license plates can be formed if every...,"58,500",14,"[So for the first part of the license plate, w...",[{'text': 'First we need to find the number of...,1,1,12
445,446,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-28T07:53:49.057201,Evaluate $\log_264$.,6,5,"[First, let's think about what this problem is...","[{'text': 'Ok, let's first break down $264$ in...",3,0,2


In [26]:
get_total_steps_sum_each_rating(clean_df_option_3)

{'total_neg_1': np.int64(280),
 'total_zero': np.int64(104),
 'total_pos_1': np.int64(931)}

In [27]:
clean_df_option_3.to_csv("datasets/sample/prm800k-03-algo3-clean.csv", index=False)